# 对TF-IDF降维处理

## 导包与读入数据

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import jieba.posseg as pseg

import jieba
import re
from sklearn.model_selection import StratifiedKFold,KFold
import random

import fasttext
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn import linear_model

In [ ]:
# 读取训练和预测数据
data_path = 'data/train_second.csv'
df = pd.read_csv(data_path,header = 0, encoding='utf8')

df2 = pd.read_csv('data/train_first.csv', header = 0, encoding='utf8')

df = pd.concat([df, df2], ignore_index=True)

test_data_path = 'data/predict_second.csv'
test_df = pd.read_csv(test_data_path,header = 0, encoding='utf8')

# 数据清洗

In [ ]:
# 训练集去重
df.drop_duplicates(subset='Discuss', keep='last',inplace=True)
len(df)

In [ ]:
# 加载停用词
stop_word = []
stop_words_path = 'dict/stopWordList.txt'

with open(stop_words_path,encoding='utf8') as f:
    for line in f.readlines():
        stop_word.append(line.strip())
stop_word.append(' ')
# 加载情感词
dict_path = 'dict/dict.dat'
jieba.load_userdict(dict_path)
# 字符串清洗，去除停用词
def clean_str(stri):    
    stri = re.sub(u'[\s]+|[^\u4e00-\u9fa5A-Za-z]+|<br />','',stri)
#     stri = re.sub(r'<br />|[\s+\.\!\/_\-,$%^*(+\"\']+|[+—【】！，。？、～~@#￥%……&*（）]|[0-9]+', ' ', stri)  # 正则替换

    cut_str = jieba.cut(stri.strip())
    list_str = [word for word in cut_str if word not in stop_word]
    stri = ' '.join(list_str)
    return stri

df['Discuss'] = df['Discuss'].map(lambda x : clean_str(x))
test_df['Discuss'] = test_df['Discuss'].map(lambda x : clean_str(x))

In [ ]:
# 空白的处理方式
def fillnull(x):
    if x == '':
        return '_na_'
    else:
        return x
    
df['Discuss'] = df['Discuss'].map(lambda x: fillnull(x))
test_df['Discuss'] = test_df['Discuss'].map(lambda x: fillnull(x))

# 辅助函数，评测函数


In [ ]:
# 构造fasttext使用的文本
def fasttext_data(data,label):
    fasttext_data = []
    for i in range(len(label)):
        sent = data[i]+"\t__label__"+str(int(label[i]))
        fasttext_data.append(sent)
    with open('train.txt','w') as f:
        for data in fasttext_data:
            f.write(data)
            f.write('\n')
    return 'train.txt'

# 得到预测值
def get_predict(pred):
    score = np.array([1,2,3,4,5])
    pred2 = []
    for p in pred:
        pr = np.sum(p * score)
        pred2.append(pr)
    return np.array(pred2)

# 评测函数
def rmsel(true_label,pred):
    true_label = np.array(true_label)
    pred = np.array(pred)  
    n = len(true_label)
    a = true_label - pred
    rmse = np.sqrt(np.sum(a * a)/n)
    b = 1/(1+rmse)
    return b

# 交叉验证函数

In [ ]:
# 朴素贝叶斯、逻辑回归模型融合
def lrnb_cv(model1, model2, model3, df, test_df, train_merge):
    df = df.sample(frac=1)  # 对行做shuffle
    df = df.reset_index(drop=True)
#     # tf-idf向量,目前min_df=1效果最好
#     vec = TfidfVectorizer(ngram_range=(1,2), min_df=1, max_df=0.8,use_idf=1, smooth_idf=1, sublinear_tf=1)
#     trn_term_doc = vec.fit_transform(df['Discuss'])
#     test_term_doc = vec.transform(test_df['Discuss'])
#     # tf-idf降维
#     tsvd = TruncatedSVD(n_components = 180)
#     trn_term_doc = tsvd.fit_transform(trn_term_doc)
#     test_term_doc = tvsd.transform(test_term_doc)
    
    # 取出模型，lr_model和nb_model
    nb_model = model1
    lr_model = model2
    ri_model = model3
    X = trn_term_doc_scale
    y = df['Score'].values
    lr_pred, nb_pred ,ri_pred= [],[],[]
    folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X,y))
    
    for train_index, test_index in folds:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # 朴素贝叶斯训练
        nb_model.fit(X_train, y_train)
        pred_i = nb_model.predict_proba(X_test)
        pred_i = get_predict(pred_i)
        print('nb cv:', rmsel(y_test, pred_i))
        train_merge.loc[test_index, 'nb'] = pred_i  # 将验证集nb预测值进行存储
        train_merge.loc[test_index, 'score1'] = y_test  # 将验证集实际结果进行存储
        # 逻辑回归训练
        lr_model.fit(X_train, y_train)
        pred_i = lr_model.predict_proba(X_test)
        pred_i = get_predict(pred_i)
        print('lr cv:', rmsel(y_test, pred_i))
        train_merge.loc[test_index, 'lr'] = pred_i  # 将验证集lr预测值进行存储
        train_merge.loc[test_index, 'score2'] = y_test  # 将验证集实际结果进行存储
        # 岭回归训练
        ri_model.fit(X_train, y_train)
        pred_i = ri_model.predict(X_test)
        print('ri cv:', rmsel(y_test, pred_i))
        train_merge.loc[test_index, 'ri'] = pred_i  # 将验证集ridge预测值进行存储
        train_merge.loc[test_index, 'score4'] = y_test  # 将验证集实际结果进行存储
        
        # 朴素贝叶斯预测
        nb_predi = nb_model.predict_proba(test_term_doc_scale)
        nb_predi = get_predict(nb_predi)
        nb_pred.append(nb_predi)
        # 逻辑回归预测
        lr_predi = lr_model.predict_proba(test_term_doc_scale)
        lr_predi = get_predict(lr_predi)
        lr_pred.append(lr_predi)
        # 岭回归预测
        ri_predi = ri_model.predict(test_term_doc_scale)
        ri_pred.append(ri_predi)
    nb_pred = np.array(nb_pred)
    nb_pred = np.mean(nb_pred, axis=0)
    lr_pred = np.array(lr_pred)
    lr_pred = np.mean(lr_pred, axis=0)
    ri_pred = np.array(ri_pred)
    ri_pred = np.mean(ri_pred, axis=0)
    return nb_pred, lr_pred, ri_pred # 返回三个模型预测结果

In [ ]:
# fasttext模型
def fast_cv(df, test_df, train_merge):
#     df = df.sample(frac=1,random_state=2018)  # 对行做shuffle
#     df = df.reset_index(drop=True)
    fast_pred = []
    folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X, y))
    rmsels = []
    for train_index, test_index in folds:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_file = fasttext_data(X_train,y_train)
        # fasttext训练
        classifier = fasttext.supervised(train_file, 'model.model', lr=0.08, dim=256, word_ngrams=3,bucket=200000,loss='hs', label_prefix="__label__")
        result = classifier.predict_proba(df.loc[test_index,'Discuss'].tolist(),k=5)
        pred = [[int(sco) * proba for sco,proba in result_i] for result_i in result]
        pred = [sum(pred_i) for pred_i in pred]
        print('fast cv:',rmsel(y_test,pred))
        train_merge.loc[test_index, 'fast'] = pred  # 将验证集fasttext预测值进行存储
        train_merge.loc[test_index, 'score3'] = y_test  # 将验证集实际结果进行存储
        # fasttext预测
        test_result = classifier.predict_proba(test_df['Discuss'].tolist(),k=5)
        fast_predi = [[int(sco) * proba for sco,proba in result_i] for result_i in test_result]
        fast_predi = [sum(pred_i) for pred_i in fast_predi]
        fast_pred.append(fast_predi)
        
    fast_pred = np.array(fast_pred)
    fast_pred = np.mean(fast_pred, axis=0)
    return fast_pred  # 返回fasttext模型预测结果

# TF-IDF

In [ ]:
from sklearn.decomposition import TruncatedSVD
vec = TfidfVectorizer(ngram_range=(1,2), min_df=1, max_df=0.8,use_idf=1, smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(df['Discuss'])
test_term_doc = vec.transform(test_df['Discuss'])
# print(type(test_term_doc))
# tf-idf降维
tsvd = TruncatedSVD(n_components = 180)
tsvd.fit(trn_term_doc)
trn_term_doc = tsvd.transform(trn_term_doc)
test_term_doc = tsvd.transform(test_term_doc)

# 融模型

In [ ]:
nb_model = MultinomialNB()  # 朴素贝叶斯回归
lr_model = LogisticRegression(C=10, class_weight='balanced')  # 逻辑回归模型
ri_model = linear_model.Ridge() # 岭回归模型


In [ ]:
from sklearn import preprocessing
min_max = preprocessing.MinMaxScaler(feature_range=(0, 1))
trn_term_doc_scale = min_max.fit_transform(trn_term_doc)
test_term_doc_scale = min_max.transform(test_term_doc)

In [ ]:
data = np.zeros((len(df), 8))
train_merge = pd.DataFrame(data)
train_merge.columns = ['nb','lr','fast','ri','score1','score2','score3','score4']
nb_pred, lr_pred ,ri_pred= lrnb_cv(nb_model, lr_model, ri_model, df, test_df, train_merge)

In [ ]:
# fasttext模型
X = df['Discuss'].values
y = df['Score'].values
fast_pred = fast_cv(df, test_df, train_merge)

In [ ]:
# 创建测试集
data = np.zeros((len(test_df), 4))
test = pd.DataFrame(data)
feature_columns=['nb','lr', 'fast','ri']
test.columns = ['nb','lr', 'fast','ri']
test['nb'], test['lr'], test['fast'], test['ri'] =  nb_pred, lr_pred, fast_pred, ri_pred
test.describe()

# xgb调参

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [ ]:
# 数据准备
feature_columns=['nb','lr', 'fast', 'ri']
X = train_merge[feature_columns].values
y = train_merge['score1'].values


In [ ]:
def modelfit(alg, X, y, useTrainCV=True, early_stopping_rounds=50, cv_folds=5, printFeatureImportance=True):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X, y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds, 
                          metrics='rmse', early_stopping_rounds= early_stopping_rounds)
        alg.set_params(n_estimators = cvresult.shape[0])
#         print(cvresult)
        print(cvresult.shape[0])
    #Fit the algorithm on the data
#     alg.fit(X, y, eval_metric='rmse')
    
    #Predict training set:
#     y = alg.predict(X)
#     dtrain_predprob = alg.predict_proba(X)[:,]

    #Print model report:
#     print("RMSE : %.4g" % metrics.mean_squared_error(X, y))

#     print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))

    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [ ]:
# 得到学习熟虑为0.1时的理想决策树目
import xgboost as xgb
xgb1 = xgb.XGBRegressor(learning_rate=0.1,n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
                      objective='reg:linear', eval_metric='rmse', scale_pos_weight=1, seed=2018)
modelfit(xgb1, X, y)

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
score = make_scorer(rmsel)
params_test1 = {'max_depth': list(range(3,8,2)), 'min_child_weight': list(range(1,6,2))}
xgb2 = xgb.XGBRegressor(learning_rate=0.1,n_estimators=110, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
                      objective='reg:linear', eval_metric='rmse', scale_pos_weight=1, seed=2018)
gsearch1 = GridSearchCV(estimator=xgb2, param_grid=params_test1, scoring=score, cv=5)
gsearch1.fit(X, y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
import xgboost as xgb
bst = xgb.XGBRegressor(learning_rate=0.01,n_estimators=1100, max_depth=5, min_child_weight=5, gamma=0, subsample=0.8,
                       eval_metric='rmse', scale_pos_weight=1, seed=2018)
xgb_pred = []
folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X,y))
es = []
for tr_index, te_index in folds:
    X_train, X_test = X[tr_index], X[te_index]
    y_train, y_test = y[tr_index], y[te_index]
    bst.fit(X_train, y_train)
    y_pred = bst.predict(X_test)
    e = rmsel(y_test, y_pred)
    print(e)
    
    test_pred = bst.predict(test[feature_columns].values)
    xgb_pred.append(test_pred)
    es.append(e)
print(np.mean(es, axis=0)) 
 
xgb_pred = np.array(xgb_pred)
xgb_pred = np.mean(xgb_pred, axis=0)

# GDBT调参

In [ ]:
gdbt_model = GradientBoostingRegressor(learning_rate=0.01,n_estimators=1500, min_samples_split=1000,min_samples_leaf=30,max_depth=5,max_features='auto',subsample=0.8,random_state=2018)

gdbt_pred = []
folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X,y))
es = []
for tr_index, te_index in folds:
    X_train, X_test = X[tr_index], X[te_index]
    y_train, y_test = y[tr_index], y[te_index]
    gdbt_model.fit(X_train, y_train)
    y_pred = gdbt_model.predict(X_test)
    e = rmsel(y_test, y_pred)
    print(e)
    
    test_pred = gdbt_model.predict(test[feature_columns].values)
    gdbt_pred.append(test_pred)
    es.append(e)
print(np.mean(es, axis=0)) 
gdbt_pred = np.array(gdbt_pred)
gdbt_pred = np.mean(gdbt_pred, axis=0)

# 输出结果

In [ ]:
np.percentile(xgb_pred, 0.01),np.percentile(xgb_pred, 0.015),np.percentile(xgb_pred, 5),np.percentile(xgb_pred, 20),np.percentile(xgb_pred, 30),np.percentile(xgb_pred, 62),np.percentile(xgb_pred, 70)

In [ ]:
xgb_pred2 = xgb_pred
# xgb_pred1 = np.where(xgb_pred1<1.72, 1,xgb_pred1)
xgb_pred2 = np.where((xgb_pred2>1.305)&(xgb_pred2<1.28), 2, xgb_pred2)
xgb_pred2 = np.where((xgb_pred2<3.3)&(xgb_pred2>1.99), 3, xgb_pred2)
xgb_pred2 = np.where((xgb_pred2<4.1)&(xgb_pred2>3.9), 4, xgb_pred2)
xgb_pred2 = np.where(xgb_pred2>4.61, 5, xgb_pred2)

In [ ]:
test['Id'] = test_df['Id']
test[['Id', 'xgb_merge2']].to_csv('result/0326-3.csv',index=None,header =None)